**作业题：**
业务背景和价值说明：

项目的实战背景是二家超市的真实数据，数据时间段是从 20160101 到 20171031。

超市的主要商业场景

通过对比分析，来发现会员与非会员的价值区别

构建会员标签，进行会员的精准营销

商业知识点思考：

会员对企业实现数据化经营（特别是线下）是核心

如何对会员数据进行更深入数据分析，不断打会员打上标签，从而为相关业务部门提供决策参考
SQL 知识点：
Case when

子查询

变量定义

取 2017 年 8 月份，会员的消费特征分析，对比会员 VS 非会员：
1、 平均客单价
2、 不同客单价区间订单，销售金额分布
【100 以下，以 20 为单位划分，100-200 按 50划分，200 以上这几个区间段】

3、 订单购买的商品数分布【每个订单中，购买 1，2，3，4，5，。。。件】

4、 周末 VS 非周末 中的会员 VS 非会员 的订单量占比分布；


In [21]:
import pymysql  
import  pymysql.cursors 
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import text 
engine=create_engine('mysql+pymysql://用户名:密码@IP:端口号/数据库名称')
#https://blog.csdn.net/cugb0451/article/details/45665307 解决百分号报错问题

In [22]:
sql = text(''' -- 查看
select * from demo.OrderItem t limit 8;'''
)
df = pd.read_sql_query(sql,engine)
df

,SDate,ShopID,SheetID,GoodsID,CateID,Qty,CostValue,SaleValue,OriSaleValue,Cost,Price
0,20160101,WDGC,20160101WDGCP00210000331,30013844,914110000,1.000,4.160000,5.520,5.520,4.160000,5.52
1,20160101,WDGC,20160101WDGCP00210000332,30009715,912060201,0.139,4.465525,6.047,6.047,32.126082,43.50
2,20160101,WDGC,20160101WDGCP00210000337,30020251,923000002,0.867,3.694870,4.861,4.861,4.261676,5.61
3,20160101,WDGC,20160101WDGCP00210000338,30009254,912060400,0.585,6.095695,6.688,11.368,10.419991,11.43
4,20160101,WDGC,20160101WDGCP00210000340,30009254,912060400,1.042,10.857631,11.904,20.244,10.419991,11.42
5,20160101,WDGC,20160101WDGCP00210000343,30006730,915000106,0.538,20.490351,22.286,28.636,38.086154,41.42
6,20160101,WDGC,20160101WDGCP00210000343,30009254,912060400,2.567,26.748117,29.325,49.865,10.419991,11.42
7,20160101,WDGC,20160101WDGCP00210000343,30149952,913050103,1.000,7.320000,7.520,9.920,7.320000,7.52


In [23]:
sql = text('''-- 平均客单价
    select case when t.MemberID is null then '非会员' else '会员' end as
    member_type
    ,count(distinct t.SheetID) as order_number
    ,sum(t.PayValue) as pay_value
    ,sum(t.PayValue)/count(distinct t.SheetID) as avg_order
    from demo.OrderList t
    where t.SDate between '20170801' and '20170831'
    and t.ShopID='CDLG'
    group by 
    case when t.MemberID is null then '非会员' else '会员' end''')
df = pd.read_sql_query(sql,engine)
df

,member_type,order_number,pay_value,avg_order
0,会员,13806,534316.2,38.701738
1,非会员,40228,1887324.0,46.915681


In [13]:
sql = text(''' -- 客单价区间订单
select case when t.MemberID is null then '非会员' else '会员' end as member_type
,case when t.PayValue<=20 then '0-20]'
when t.PayValue>20 and t.PayValue<=40 then '(20-40]'
when t.PayValue>40 and t.PayValue<=60 then '(40-60]'
when t.PayValue>60 and t.PayValue<=80 then '(60-80]'
when t.PayValue>80 and t.PayValue<=100 then '(80-100]'
when t.PayValue>100 and t.PayValue<=150 then '(100-150]'
when t.PayValue>150 and t.PayValue<=200 then '(150-200]'
when t.PayValue>200 then '(200-]'
end as order_type
,count(distinct t.SheetID) as order_number
,sum(t.PayValue) as pay_value
from demo.OrderList t
where t.SDate between '20170801' and '20170831'
and t.ShopID='CDLG'
group by case when t.MemberID is null then '非会员' else '会员' end
,case when t.PayValue<=20 then '0-20]'
when t.PayValue>20 and t.PayValue<=40 then '(20-40]'
when t.PayValue>40 and t.PayValue<=60 then '(40-60]'
when t.PayValue>60 and t.PayValue<=80 then '(60-80]'
when t.PayValue>80 and t.PayValue<=100 then '(80-100]'
when t.PayValue>100 and t.PayValue<=150 then '(100-150]'
when t.PayValue>150 and t.PayValue<=200 then '(150-200]'
when t.PayValue>200 then '(200-]'
end;'''
)
df = pd.read_sql_query(sql,engine)
df

,member_type,order_type,order_number,pay_value
0,会员,(100-150],608,73079.2
1,会员,(150-200],200,34214.2
2,会员,(20-40],3530,101967.7
3,会员,(200-],132,46423.4
4,会员,(40-60],1824,89973.5
5,会员,(60-80],1089,75416.9
6,会员,(80-100],582,52184.8
7,会员,0-20],5841,61056.5
8,非会员,(100-150],3362,361800.8
9,非会员,(150-200],485,90187.8


In [15]:
sql = text(''' -- 订单购买的商品数分布【每个订单中，购买1，2，3，4，5，。。。件】
select t.Qty
,case when t1.MemberID is null then '非会员' else '会员' end
as member_type
,count(distinct t1.SheetID) as order_nubmer
from demo.OrderItem t
join demo.OrderList t1 on t.SheetID = t1.SheetID
where t.SDate between '20170801' and '20170831'
and t.ShopID='CDLG'
group by t.Qty
,case when t1.MemberID is null then '非会员' else '会员' end;'''
)
df = pd.read_sql_query(sql,engine)
df

,Qty,member_type,order_nubmer
0,-27.793,非会员,1
1,-25.000,非会员,1
2,-24.000,会员,3
3,-20.000,非会员,1
4,-12.000,非会员,1
5,-6.116,非会员,1
6,-5.732,会员,1
7,-4.348,会员,1
8,-4.000,非会员,2
9,-3.710,非会员,1


In [19]:
sql = text(''' select w_day
,member_type
,order_number
,order_number/@total_order as per
from
(select case when weekday(SDate) in (5,6) then '周末' else '非周末' end
as w_day
,case when t.MemberID is null then '非会员' else '会员' end as
member_type
,count(distinct t.SheetID) as order_number
from demo.OrderList t
where t.SDate between '20170801' and '20170831'
and t.ShopID='CDLG'
group by case when weekday(SDate) in (5,6) then '周末' else '非周末' end
,case when t.MemberID is null then '非会员' else '会员' end) t1
join (select @total_order :=count(distinct t.SheetID)
from demo.OrderList t
where t.SDate between '20170801' and '20170831'
and t.ShopID='CDLG') t2;'''
)
df = pd.read_sql_query(sql,engine)
df

C:\Users\lining\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1287, "Setting user variables within expressions is deprecated and will be removed in a future release. Consider alternatives: 'SET variable=expression, ...', or 'SELECT expression(s) INTO variables(s)'.")
  result = self._query(query)


,w_day,member_type,order_number,per
0,周末,会员,3676,0.068031
1,周末,非会员,11128,0.205944
2,非周末,会员,10130,0.187475
3,非周末,非会员,29100,0.538550
